In [1]:
import pandas as pd 

In [2]:
import urllib.request

#url = 'http://data.phishtank.com/data/'
#urllib.request.urlretrieve(url, "online-valid.csv")

#print('Download complete')

<font color="Red"><h1>Phish urls</h1></font>

In [3]:
data1 = pd.read_csv('dataset/online-valid.csv')
data1.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6557033,http://u1047531.cp.regruhosting.ru/acces-inges...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:43+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
1,6557032,http://hoysalacreations.com/wp-content/plugins...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:37+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
2,6557011,http://www.accsystemprblemhelp.site/checkpoint...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:54:31+00:00,yes,2020-05-09T21:55:38+00:00,yes,Facebook
3,6557010,http://www.accsystemprblemhelp.site/login_atte...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:53:48+00:00,yes,2020-05-09T21:54:34+00:00,yes,Facebook
4,6557009,https://firebasestorage.googleapis.com/v0/b/so...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:49:27+00:00,yes,2020-05-09T21:51:24+00:00,yes,Microsoft


In [4]:
data1.shape

(14858, 8)

The above dataset contains 14858 rows of phishing sites so for this project i m considering the 5k rows of phishing url and 5k rows of legitmate urls to avoid biasness

In [5]:
phishurl = data1.sample(n = 5000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6514946,http://confirmprofileaccount.com/,http://www.phishtank.com/phish_detail.php?phis...,2020-04-19T11:06:55+00:00,yes,2020-04-19T13:42:41+00:00,yes,Other
1,4927651,http://www.marreme.com/MasterAdmin/04mop.html,http://www.phishtank.com/phish_detail.php?phis...,2017-04-04T19:35:54+00:00,yes,2017-05-03T23:00:42+00:00,yes,Other
2,5116976,http://modsecpaststudents.com/review/,http://www.phishtank.com/phish_detail.php?phis...,2017-07-25T18:48:30+00:00,yes,2017-07-28T16:01:36+00:00,yes,Other
3,6356131,https://docs.google.com/forms/d/e/1FAIpQLScL6L...,http://www.phishtank.com/phish_detail.php?phis...,2020-01-13T20:13:37+00:00,yes,2020-01-17T01:55:38+00:00,yes,Other
4,6535965,https://oportunidadedasemana.com/americanas//?...,http://www.phishtank.com/phish_detail.php?phis...,2020-04-29T00:01:03+00:00,yes,2020-05-01T10:55:35+00:00,yes,Other


In [6]:
phishurl.shape

(5000, 8)

<font color="Red"><h1>Legitmate urls</h1></font>

In [7]:
data2 = pd.read_csv("dataset/Benign_list_big_final.csv")
data2.columns = ['URLs']
data2.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [8]:
legiurl = data2.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [9]:
legiurl.shape

(5000, 1)

In this way we created 5k phish urls and 5k legiurl

<font color="Red"><h1 >Feature Extraction</h1></font>

So now trying to extract features from the above url datasets
<br>
And the extracted features are categorized into 3 parts<br>
<li>Domain based Features</li>
<li>HTML & Javascript based Features</li>
<li>Address Bar based Features</li>


<font color="blue"><h3> 1.Domain based features</h3></font>
Out of multiple domain based features for our project we are considering three best features<br>

<li>Age of Domain</li>
<li>End Period of Domain</li>
<li>DNS Record</li>

In [10]:
!pip install python-whois


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<font color="green"><h4>B. DNS record</h4></font>

DNS record is empty or not found then, <br>the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

<font color="green"><h4>C. Age of Domain</h4> </font>

Most of the phishing websites live shorter.
The minimum age for legitmate website considered for this website is considered at least 12 months.
less than 12 months means phishing(1) and more means legitmate(0).

In [11]:
#The difference between termination time and creation time (Domain_Age)

def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age

<font color="green"><h4>D. End period of domain</h4></font>

For this feature, <br>the remaining domain time is calculated by finding the different between expiration time & current time.<br>
If end period of domain > 6 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [12]:
#The difference between termination time and current time (Domain_End) 
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end

<font color="blue"> <h3>2. Address Bar Features</h3></font>

<li>Domain</li>
<li>IP Adress</li>
<li>@ - symbol in url</li>
<li>Length of url</li>
<li>Depth of url</li>
<li>URL shortening service(tiny URL)</li>
<li>"-" in URL</li>
<li>"//" redirect</li>


In [13]:
# importing required packages for this section
from urllib.parse import urlparse,urlencode
import ipaddress
import re

<font color="green"><h4>A. Domain of the URL</h4></font>

In [14]:
def getDomain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain

<font color="green"><h4>B. "//" in url</h4></font>

If URl start with http the "//" at the 5th position<br>
If URL start with https then "//" at the 6th position.

If the "//" is anywhere in the URL apart from after the protocal, 
<br>the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [15]:
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

<font color="green"> <h4>C. URL shortening service(tiny URL)</h4></font>

If the URL is using Shortening Services, <br>the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [16]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [17]:
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

<font color="green"><h4>D. IP adress</h4></font>

If the domain part of URL has IP address,<br> the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [18]:
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip


<font color="green"><h4>E.  "@" Symbol in URL</h4></font>

“@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and <br>the real address often follows the “@” symbol.

If the URL has '@' symbol, <br>the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [19]:
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

<font color="green"><h4>F. '-' in URL</h4></font>

Phishers tend to add prefixes or suffixes separated by (-) to the domain name <br>so that users feel that they are dealing with a legitimate webpage.
<br>
If the URL has '-' symbol in the domain part of the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).


In [20]:
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0     

<font color="green"><h4>G. Length of url</h4></font>
<br>
In this project, if the length of the URL is greater than or equal 54 characters <br>then the URL classified as phishing otherwise legitimate.

In [21]:
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

<font color="green"><h4>H. Depth of url</h4></font>
<br>
This feature calculates the number of sub pages in the given url based on the '/'.
<br>
The value of feature is a numerical based on the URL.

In [22]:
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

<font color="REd"><h1>Computing URL features</h1></font>

Creating the list and a function that calls other functions and stores all the feature or the url in the list.<br>
I m extracting the feature of each url and appending it to the list.

In [ ]:
def featureExtraction(url, lablel):

    features = []
    #Adress bar based features - 8 features
    features.append(getDomain(url))
    features.append(havingIP(url))
    features.append(haveAtSign(url))
    features.append(prefixSuffix(url))
    features.append(tinyURL(url))
    features.append(redirection(url))
    features.append(getDepth(url))
    features.append(getLength(url))

    #Domain based features(dbf) - 3 features
    dbf = 0
    try:
        domain_name = whois.whois(urlparse(url).netloc)
    except:
        dbf = 1

    features.append(dbf)
    features.append(1 if dbf ==1 else domainAge(domain_name))
    features.append(1 if dbf == 1 else domainEnd(domain_name))

    return features
    

<font color="blue"><h2>Legitmate feature extraction</h2></font>

In [24]:
legiurl.shape

(5000, 1)

In [25]:
print(legiurl.columns)

Index(['URLs'], dtype='object')


In [26]:
legitmateURL_features = []
label = 0

for i in range(0, 5000):
    url = legiurl['URLs'][i]
    legitmateURL_features.append(featureExtraction(url, label)+[label])

In [27]:
print(legiurl.columns)

Index(['URLs'], dtype='object')


In [28]:
#convert the list into dataframe

feature_names = ['Domain', 'Have_IP', 'Have_At', 'Prefix&Suffix', 'TinyURL', 'Redirection', 'URL_length', 'URL_Depth', 'DNS_Record', 'Domain_Age', 'Domain_End','Label']



In [29]:
legitmate = pd.DataFrame(legitmateURL_features, columns = feature_names)
legitmate.head()

,Domain,Have_IP,Have_At,Prefix&Suffix,TinyURL,Redirection,URL_length,URL_Depth,DNS_Record,Domain_Age,Domain_End,Label
0,graphicriver.net,0,0,0,0,0,1,1,1,1,1,0
1,ecnavi.jp,0,0,0,0,1,1,1,1,1,1,0
2,hubpages.com,0,0,0,0,0,1,1,1,1,1,0
3,extratorrent.cc,0,0,0,0,0,3,1,1,1,1,0
4,icicibank.com,0,0,0,0,0,3,1,1,1,1,0


In [30]:
legitmate.to_csv('dataset/legitmate_dataset.csv', index = False)
#store the extracted legitmate url features to csv file

<font color='blue'> <h2>Phishing url feature extraction</h2></font>

In [31]:
phishurl.shape

(5000, 8)

In [32]:
phish_features = []
label = 1
for i in range(0, 5000):
    url = phishurl['url'][i]
    phish_features.append(featureExtraction(url, label) + [label])

In [ ]:
feature_names = ['Domain', 'Have_IP', 'Have_At', 'Prefix&Suffix', 'TinyURL', 'Redirection',
                  'URL_length', 'URL_Depth', 'DNS_Record', 'Domain_Age', 'Domain_End',
                    'Label']

phishing = pd.DataFrame(phish_features, columns = feature_names)
phishing.head()

,Domain,Have_IP,Have_At,Prefix&Suffix,TinyURL,Redirection,URL_length,URL_Depth,DNS_Record,Domain_Age,Domain_End,Label
0,confirmprofileaccount.com,0,0,0,1,0,0,0,1,1,1,1
1,marreme.com,0,0,0,0,0,2,0,1,1,1,1
2,modsecpaststudents.com,0,0,0,0,0,1,0,1,1,1,1
3,docs.google.com,0,0,0,0,0,5,1,1,1,1,1
4,oportunidadedasemana.com,0,0,0,0,1,1,1,1,1,1,1


In [34]:
#storing the extracted features for phishin urls
phishing.to_csv('dataset/phishing.csv', index = False)

<font color='red'><h2>Combining the dataset for final model building purpose</h2></font>

In [35]:
combinedDataset = pd.concat([legitmate, phishing]).reset_index(drop = True)
combinedDataset.head()

,Domain,Have_IP,Have_At,Prefix&Suffix,TinyURL,Redirection,URL_length,URL_Depth,DNS_Record,Domain_Age,Domain_End,Label
0,graphicriver.net,0,0,0,0,0,1,1,1,1,1,0
1,ecnavi.jp,0,0,0,0,1,1,1,1,1,1,0
2,hubpages.com,0,0,0,0,0,1,1,1,1,1,0
3,extratorrent.cc,0,0,0,0,0,3,1,1,1,1,0
4,icicibank.com,0,0,0,0,0,3,1,1,1,1,0


In [36]:
combinedDataset.shape

(10000, 12)

In [37]:
combinedDataset.to_csv('dataset/CombinedDataset.csv', index=False)